In [1]:
# ======================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# Don't use lambda layers in your model.
# You do not need them to solve the question.
# Lambda layers are not supported by the grading infrastructure.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ======================================================================
#
# NLP QUESTION
#
# Build and train a classifier for the sarcasm dataset.
# The classifier should have a final layer with 1 neuron activated by sigmoid as shown.
# It will be tested against a number of sentences that the network hasn't previously seen
# and you will be scored on whether sarcasm was correctly detected in those sentences.

In [2]:
%pip install json
%pip install tensorflow
%pip install numpy
%pip install urllib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tensorflow-2.14.0-cp311-cp311-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl (284.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\samapant\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\tensorflow\\python\\keras\\optimizer_v2\\adagrad.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    # Load the JSON file
    with open("./sarcasm.json", 'r') as f:
        datastore = json.load(f)

# Collect sentences and labels into the lists
    # DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    sentences = []
    labels = []
    # YOUR CODE HERE
    for item in datastore:
      sentences.append(item['headline'])
      labels.append(item['is_sarcastic'])

    # Split the sentences
    training_sentences = sentences[:training_size]
    testing_sentences = sentences[training_size:]

    # Split the labels
    training_labels = labels[:training_size]
    testing_labels = labels[training_size:]

    # Initialize the Tokenizer class
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

    # Generate the word index dictionary
    tokenizer.fit_on_texts(training_sentences)
    word_index = tokenizer.word_index

    # Generate and pad the training sequences
    training_sequences = tokenizer.texts_to_sequences(training_sentences)
    training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Generate and pad the testing sequences
    testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Convert the labels lists into numpy arrays
    training_labels = np.array(training_labels)
    testing_labels = np.array(testing_labels)

    lstm_dim = 32
    dense_dim = 24

    model = tf.keras.Sequential([
        # YOUR CODE HERE. KEEP THIS OUTPUT LAYER INTACT OR TESTS MAY FAIL
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
        tf.keras.layers.Dense(dense_dim, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Accuracy 95%
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.95:
                print("\nReached 99.5% accuracy so cancelling training!")
                self.model.stop_training = True
    callbacks = myCallback()

    # Checkpoint
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="Question_4.h5",
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
        )

    model.fit(training_padded, training_labels,
              validation_data=(testing_padded, testing_labels),
              epochs=500,
              callbacks=[callbacks, checkpoint])

    return model 

In [4]:
# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("Question_4.h5")

Epoch 1/500
625/625 [==============================] - ETA: 0s - loss: 0.4547 - accuracy: 0.7713
Epoch 1: val_accuracy improved from -inf to 0.81860, saving model to Question_4.h5
625/625 [==============================] - 38s 55ms/step - loss: 0.4547 - accuracy: 0.7713 - val_loss: 0.3911 - val_accuracy: 0.8186
Epoch 2/500
624/625 [============================>.] - ETA: 0s - loss: 0.3483 - accuracy: 0.8399
Epoch 2: val_accuracy did not improve from 0.81860
625/625 [==============================] - 46s 73ms/step - loss: 0.3483 - accuracy: 0.8400 - val_loss: 0.3927 - val_accuracy: 0.8149
Epoch 3/500
624/625 [============================>.] - ETA: 0s - loss: 0.3246 - accuracy: 0.8561
Epoch 3: val_accuracy improved from 0.81860 to 0.82546, saving model to Question_4.h5
625/625 [==============================] - 43s 68ms/step - loss: 0.3245 - accuracy: 0.8562 - val_loss: 0.3747 - val_accuracy: 0.8255
Epoch 4/500
624/625 [============================>.] - ETA: 0s - loss: 0.3132 - accuracy: 

c:\Users\samapant\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
